# Quantarhei multilevel molecules and aggregates

In [1]:
from quantarhei import Molecule
from quantarhei import energy_units
import numpy as np

Molecule with single excited state per excitation band

In [2]:
# Molecule definition - single excited state per excitation band
with energy_units("1/cm"):
    mol = Molecule(name="Carotenoid single ex.",elenergies=[0,18900,34985])  # excitation energies in 1/cm
    mol.set_dipole(0,1,[1.28 , 0.0, 0.0])  # 0->1 transition dipole in Debye
    mol.set_dipole(1,2,[0.90 , 0.0, 0.0])  # 1->2 transition dipole in Debye
    # for the rest of the transitions assume zero dipole moment
    mol._position = np.array([0.0,0.0,0.0],dtype="f8")   # center of the molecule in Angstroms
    
# print molecular information
print("Number of electronic bands:",mol.Nb.size)
print("Number of electronic states:",mol.nel)
print("Number of electronic states per electronic band:",mol.Nb)
print("Correspondence of electronic states to electronic bands:",mol.which_band)

Number of electronic bands: 3
Number of electronic states: 3
Number of electronic states per electronic band: [1 1 1]
Correspondence of electronic states to electronic bands: [0 1 2]


Molecule with multiple excited states per excitation band

In [3]:
# Molecule definition - multiple excited states per excitation band
with energy_units("1/cm"):
    mol2 = Molecule(name="Carotenoid single ex.",elenergies=[0,[17400,18900],[30060,34985]])  # excitation energies in 1/cm
    mol2.set_dipole(0,1,[0.0, 0.03, 0.0])  # 0->1 transition dipole in Debye (S0->S1)
    mol2.set_dipole(0,2,[1.28 , 0.0, 0.0]) # 0->2 transition dipole in Debye (S0->S2)
    mol2.set_dipole(1,3,[0.0, 1.5, 0.0])   # 1->3 transition dipole in Debye (S1->Sn)
    mol2.set_dipole(2,4,[0.90 , 0.0, 0.0]) # 1->2 transition dipole in Debye (S2->SN)
    # for the rest of the transitions assume zero dipole moment
    mol2._position = np.array([10.0,0.0,0.0],dtype="f8")   # center of the molecule in Angstroms
    
# print molecular information
print("Number of electronic bands:",mol2.Nb.size)
print("Number of electronic states:",mol2.nel)
print("Number of electronic states per electronic band:",mol2.Nb)
print("Correspondence of electronic states to electronic bands:",mol2.which_band)

Number of electronic bands: 3
Number of electronic states: 5
Number of electronic states per electronic band: [1 2 2]
Correspondence of electronic states to electronic bands: [0 1 1 2 2]


Set up an aggregate

In [4]:
from quantarhei import Aggregate

# Define aggregate and add individual monomers
aggreg = Aggregate()
aggreg.add_Molecule(mol)
aggreg.add_Molecule(mol2)
mult = 2 # Maximum excitation band

# Define dipole dipole coupling between monomers
aggreg.set_coupling_by_dipole_dipole_vec(epsr=1.0)
    
# set interaction energy between two transitions
#set_resonance_coupling_vec(self, i, j, coupling)
#_set_coupling_vec(self,mon1,init1,fin1,mon2,init2,fin2,coupling)

Print aggregate info even before build

In [5]:
# print vibronic states of the aggregate
print("Aggregate vibronic states and corresponding bands")
print("--------------------------------------------------")
for a, s1 in aggreg.allstates(mult=mult):
    print(a,"elsign:",s1.elstate.elsignature,"vibsign:",s1.vsig,"elindex:",s1.elstate.index,"band:",s1.elstate.band)


# print interaction elements
print("\nAggregate interaction elements:")
print("--------------------------------------------------")
for n in range(len(aggreg.resonance_coupling_vec)):
    coupling_element = aggreg._coupling2mol[n]
    excit1 = coupling_element[0]
    excit2 = coupling_element[1]
    print("Inter. between mon.",excit1[0]+1,"trans.",excit1[1],"->",excit1[2],
          "and mon.",excit2[0]+1,"trans.",excit2[1],"->",excit2[2],"is",
          aggreg.resonance_coupling_vec[n],"int units")

Aggregate vibronic states and corresponding bands
--------------------------------------------------
0 elsign: (0, 0) vibsign: () elindex: 0 band: 0
1 elsign: (1, 0) vibsign: () elindex: 1 band: 1
2 elsign: (0, 1) vibsign: () elindex: 2 band: 1
3 elsign: (0, 2) vibsign: () elindex: 5 band: 1
4 elsign: (2, 0) vibsign: () elindex: 3 band: 2
5 elsign: (1, 1) vibsign: () elindex: 4 band: 2
6 elsign: (1, 2) vibsign: () elindex: 6 band: 2
7 elsign: (0, 3) vibsign: () elindex: 7 band: 2
8 elsign: (0, 4) vibsign: () elindex: 8 band: 2

Aggregate interaction elements:
--------------------------------------------------
Inter. between mon. 1 trans. 0 -> 1 and mon. 2 trans. 0 -> 1 is 0.0 int units
Inter. between mon. 1 trans. 0 -> 1 and mon. 2 trans. 0 -> 2 is -0.00310723271717 int units
Inter. between mon. 1 trans. 0 -> 1 and mon. 2 trans. 0 -> 3 is 0.0 int units
Inter. between mon. 1 trans. 0 -> 1 and mon. 2 trans. 0 -> 4 is 0.0 int units
Inter. between mon. 1 trans. 0 -> 1 and mon. 2 trans. 1 -

Build aggregate

In [6]:
# Build aggregate
aggreg.build(mult=mult,sbi_for_higher_ex=False)

# Print interaction element
print(aggreg.HH[6,4],aggreg.HH[6,8])

-0.00218477300426 -0.00218477300426


## Vibronic aggregate

In [7]:
from quantarhei import Mode

# Molecule definition - single excited state per excitation band
with energy_units("1/cm"):
    mol = Molecule(name="Test molecule",elenergies=[0,[17000,18000]])  # excitation energies in 1/cm
    mol.set_dipole(0,1,[1.0 , 0.0, 0.0])  # 0->1 transition dipole in Debye
    mol.set_dipole(0,2,[1.5 , 0.0, 0.0])  # 1->2 transition dipole in Debye
    # for the rest of the transitions assume zero dipole moment
    mol._position = np.array([0.0,0.0,0.0],dtype="f8")   # center of the molecule in Angstroms
    
    # Add vibrational levels
    vib_mode = Mode(frequency = 1500.0)  # Define normal mode frequency
    mol.add_Mode(vib_mode)               # Add normal mode to the molecule
    mol.modes[0].set_shift(2,1.0)        # Define shift of the ground state and excited state potential surface
    mol.modes[0].set_shift(1,0.5)        # reorgE = frequency*(shift^2)/2  ( HuangRhys fac. = (shift**2)/2.0 )
    
# print molecular information
print("Number of electronic bands:",mol.Nb.size)
print("Number of electronic states:",mol.nel)
print("Number of electronic states per electronic band:",mol.Nb)
print("Correspondence of electronic states to electronic bands:",mol.which_band)

Number of electronic bands: 2
Number of electronic states: 3
Number of electronic states per electronic band: [1 2]
Correspondence of electronic states to electronic bands: [0 1 1]


Create copy of the first molecule and shift trasition energies

In [8]:
from copy import deepcopy
from quantarhei import convert

mol2 = deepcopy(mol)
# shift energi of the first state by +200cm-1 and the second by +100cm-1
mol2._elenergies[1:] += convert(np.array([200.0,100.0]),"1/cm","int") # energies stored in internal units
# shift position of the second molecule
mol._position = np.array([10.0,0.0,0.0],dtype="f8")   # center of the molecule in Angstroms


with energy_units("1/cm"):
    print("Old transition energies:",mol.elenergies)  # Energies are not unit managed! - this should be changed in the future
    print("New transition energies:",mol2.elenergies) # Energies are not unit managed! - this should be changed in the future
print("Transition energies in 1/cm:")
print("Old transition energies:",convert(mol.elenergies,"int","1/cm"))  
print("New transition energies:",convert(mol2.elenergies,"int","1/cm")) 
    

Old transition energies: [ 0.          3.20220766  3.39057282]
New transition energies: [ 0.          3.2398807   3.40940934]
Transition energies in 1/cm:
Old transition energies: [     0.  17000.  18000.]
New transition energies: [     0.  17200.  18100.]


Create a vibronic aggregate with multilevel molecules

In [9]:
# Define aggregate and add individual monomers
aggreg = Aggregate()
aggreg.add_Molecule(mol)
aggreg.add_Molecule(mol2)
mult = 2 # Maximum excitation band

# Define dipole dipole coupling between monomers
aggreg.set_coupling_by_dipole_dipole_vec(epsr=1.0)

Print pre-build aggregate info

In [10]:
# print vibronic states of the aggregate
print("Aggregate vibronic states and corresponding bands")
print("--------------------------------------------------")
for a, s1 in aggreg.allstates(mult=mult):
    print(a,"elsign:",s1.elstate.elsignature,"vibsign:",s1.vsig,"elindex:",s1.elstate.index,"band:",s1.elstate.band)


# print interaction elements
print("\nAggregate interaction elements:")
print("--------------------------------------------------")
for n in range(len(aggreg.resonance_coupling_vec)):
    coupling_element = aggreg._coupling2mol[n]
    excit1 = coupling_element[0]
    excit2 = coupling_element[1]
    print("Inter. between mon.",excit1[0]+1,"trans.",excit1[1],"->",excit1[2],
          "and mon.",excit2[0]+1,"trans.",excit2[1],"->",excit2[2],"is",
          aggreg.resonance_coupling_vec[n],"int units")

Aggregate vibronic states and corresponding bands
--------------------------------------------------
0 elsign: (0, 0) vibsign: (0, 0) elindex: 0 band: 0
1 elsign: (0, 0) vibsign: (0, 1) elindex: 0 band: 0
2 elsign: (0, 0) vibsign: (1, 0) elindex: 0 band: 0
3 elsign: (0, 0) vibsign: (1, 1) elindex: 0 band: 0
4 elsign: (1, 0) vibsign: (0, 0) elindex: 1 band: 1
5 elsign: (1, 0) vibsign: (0, 1) elindex: 1 band: 1
6 elsign: (1, 0) vibsign: (1, 0) elindex: 1 band: 1
7 elsign: (1, 0) vibsign: (1, 1) elindex: 1 band: 1
8 elsign: (0, 1) vibsign: (0, 0) elindex: 2 band: 1
9 elsign: (0, 1) vibsign: (0, 1) elindex: 2 band: 1
10 elsign: (0, 1) vibsign: (1, 0) elindex: 2 band: 1
11 elsign: (0, 1) vibsign: (1, 1) elindex: 2 band: 1
12 elsign: (2, 0) vibsign: (0, 0) elindex: 3 band: 1
13 elsign: (2, 0) vibsign: (0, 1) elindex: 3 band: 1
14 elsign: (2, 0) vibsign: (1, 0) elindex: 3 band: 1
15 elsign: (2, 0) vibsign: (1, 1) elindex: 3 band: 1
16 elsign: (0, 2) vibsign: (0, 0) elindex: 5 band: 1
17 elsig

Internal check of consistency for only single exited band

In [11]:
# print vibronic states of the aggregate
print("Aggregate vibronic states and corresponding bands")
print("--------------------------------------------------")
for a, s1 in aggreg.allstates(mult=1):
    print(a,"elsign:",s1.elstate.elsignature,"vibsign:",s1.vsig,"elindex:",s1.elstate.index,"band:",s1.elstate.band)

Aggregate vibronic states and corresponding bands
--------------------------------------------------
0 elsign: (0, 0) vibsign: (0, 0) elindex: 0 band: 0
1 elsign: (0, 0) vibsign: (0, 1) elindex: 0 band: 0
2 elsign: (0, 0) vibsign: (1, 0) elindex: 0 band: 0
3 elsign: (0, 0) vibsign: (1, 1) elindex: 0 band: 0
4 elsign: (1, 0) vibsign: (0, 0) elindex: 1 band: 1
5 elsign: (1, 0) vibsign: (0, 1) elindex: 1 band: 1
6 elsign: (1, 0) vibsign: (1, 0) elindex: 1 band: 1
7 elsign: (1, 0) vibsign: (1, 1) elindex: 1 band: 1
8 elsign: (0, 1) vibsign: (0, 0) elindex: 2 band: 1
9 elsign: (0, 1) vibsign: (0, 1) elindex: 2 band: 1
10 elsign: (0, 1) vibsign: (1, 0) elindex: 2 band: 1
11 elsign: (0, 1) vibsign: (1, 1) elindex: 2 band: 1
12 elsign: (2, 0) vibsign: (0, 0) elindex: 3 band: 1
13 elsign: (2, 0) vibsign: (0, 1) elindex: 3 band: 1
14 elsign: (2, 0) vibsign: (1, 0) elindex: 3 band: 1
15 elsign: (2, 0) vibsign: (1, 1) elindex: 3 band: 1
16 elsign: (0, 2) vibsign: (0, 0) elindex: 4 band: 1
17 elsig

Build the aggregate

In [12]:
# Build aggregate
aggreg.build(mult=mult,sbi_for_higher_ex=True)

In [13]:
print("Interaction between selected vibronic states")
print("--------------------------------------------")

# interaction between (1, 0) vib. (0, 0) and (0, 1) vib. (0, 0)
print("Interaction between 0_0->1_0 and 0_0->1_0")
el_coupl = aggreg.get_resonance_coupling_vec(0,0,1,1,0,1) # monomer 0 trans. 0->1 and monomer 1 transition 0->1
print("el. coupling:",el_coupl,"Franc-Condon factor:",
      aggreg.FCf[4,8]," and hamiltonian element",aggreg.HH[4,8],
      np.isclose(aggreg.HH[4,8],el_coupl*aggreg.FCf[4,8]))

# interaction between (1, 0) vib. (0, 0) and (0, 2) vib. (0, 1)
print("\nInteraction between 0_0->1_0 and 0_0->2_1")
el_coupl = aggreg.get_resonance_coupling_vec(0,0,1,1,0,2) # monomer 0 trans. 0->1 and monomer 1 transition 0->2
print("el. coupling:",el_coupl,"Franc-Condon factor:",
      aggreg.FCf[4,17]," and hamiltonian element",aggreg.HH[4,17],
      np.isclose(aggreg.HH[4,17],el_coupl*aggreg.FCf[4,17]))


# interaction between (0, 0) vib. (0, 0) and (1, 1) vib. (0, 0) 
print("\nInteraction between ground (0, 0) vib. (0, 0) and douple excited state (1,1) vib. (0,0)")
el_coupl = aggreg.get_resonance_coupling_vec(0,0,1,1,0,1) # monomer 0 trans. 0->1 and monomer 1 transition 0->2
print("el. coupling:",el_coupl,"Franc-Condon factor:",
      aggreg.FCf[0,17]," and hamiltonian element",aggreg.HH[0,20],
      np.isclose(aggreg.HH[0,20],el_coupl*aggreg.FCf[0,20]))
print("!!! Interaction between ground and double excited state so far not supported !!!")

# In this small aggregate there is no interaction between double excited states

Interaction between selected vibronic states
--------------------------------------------
Interaction between 0_0->1_0 and 0_0->1_0
el. coupling: -0.00189650434398 Franc-Condon factor: 0.882496902585  and hamiltonian element -0.0016736592093 True

Interaction between 0_0->1_0 and 0_0->2_1
el. coupling: -0.00284475651597 Franc-Condon factor: 0.51733037245  and hamiltonian element -0.00147167894794 True

Interaction between ground (0, 0) vib. (0, 0) and douple excited state (1,1) vib. (0,0)
el. coupling: -0.00189650434398 Franc-Condon factor: 0.550695314903  and hamiltonian element 0.0 False
!!! Interaction between ground and double excited state so far not supported !!!
